<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/notebooks/retriever/Query_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Query Rewrite for Advanced RAG: Pre-Retrieval Strategy**

Advanced RAG 시스템에서 검색 성능을 높이기 위한 Pre-retrieval 전략으로서 Query Rewrite을 실험적으로 탐구한다. Query Rewrite는 query를 더 풍부하거나 검색 친화적인 형태로 변형하여 더 나은 문서 검색 결과를 유도하는 것을 목표로 한다.

## **Query Rewrite Strategy**
### **Query Expansion (확장)**
- 쿼리에 의미적으로 관련된 단어를 추가하여 검색 범위를 넓힘
>"강아지 훈련" → "강아지 훈련, 반려견 교육, 개 행동 교정"

### **Query Transformation (변형)**
- 쿼리의 표현 방식을 자연어에서 키워드 중심 또는 간결한 문장으로 재구성
>"서울에서 유명한 음식은?" → "서울 맛집 음식"

### **Query Routing (라우팅)**
- 쿼리를 적절한 하위 인덱스 또는적절한 카테고리로 분기시킴
> "의료 논문 요약" → 의학 분야 인덱스로 라우팅

위 3가지 방식을 조합하여 Pre-retrieval 전략에 적용하고자 한다.

## **LLM을 이용하여 Query Rewrite**
Query Rewrite를 LLM에 맞겨 Original Query가 어떻게 더 명확하고 검색에 유리하게 재작성되는지를 관찰하고 분석하고자 한다.









In [14]:
#!pip install transformers -q

In [17]:
from transformers import pipeline
import pandas as pd

# flan-t5-base 모델 로딩
rewrite_pipe = pipeline("text2text-generation", model="google/flan-t5-base")

# 예시 쿼리 리스트 정의
queries = [
    "effect of drug abuse", # 약물 남용의 영향
    "treatment for chronic pain", # 만성 통증 치료법
    "preventive measures for hypertension", # 고혈압 예방 조치
    "diabetes in children", # 소아 당뇨병
    "symptoms of depression" #우울증의 증상
]


# 확장용 프롬프트
def expand_query(query):
    prompt = f"Expand this query with more informative keywords or related concepts: {query}"
    return rewrite_pipe(prompt, max_new_tokens=32)[0]['generated_text']

expanded_results = [(q, expand_query(q)) for q in queries]
df_expansion = pd.DataFrame(expanded_results, columns=["Original Query", "Expanded Query"])

# 변환용 프롬프트
def transform_query(query):
    prompt = f"Rewrite this query to be more helpful and specific: {query}"
    return rewrite_pipe(prompt, max_new_tokens=32)[0]['generated_text']

transformed_results = [(q, transform_query(q)) for q in queries]
df_transformation = pd.DataFrame(transformed_results, columns=["Original Query", "Transformed Query"])

# 라우팅용 프롬프트
def route_query(query):
    prompt = f"What is the most relevant domain for this query? (e.g., mental health, cardiology, addiction, pediatrics, pain management)\nQuery: {query}"
    return rewrite_pipe(prompt, max_new_tokens=16)[0]['generated_text']

routing_results = [(q, route_query(q)) for q in queries]
df_routing = pd.DataFrame(routing_results, columns=["Original Query", "Predicted Domain"])

df_all = pd.concat([
    df_expansion.set_index("Original Query"),
    df_transformation.set_index("Original Query"),
    df_routing.set_index("Original Query")
], axis=1).reset_index()
df_all



Device set to use cpu


,Original Query,Expanded Query,Transformed Query,Predicted Domain
0,effect of drug abuse,drug abuse is a common cause of death and disa...,What is the effect of drug abuse?,addiction
1,treatment for chronic pain,chronic pain is a common condition that can be...,Is there a specific treatment for chronic pain?,pain management
2,preventive measures for hypertension,hypertension is a disease that causes hyperten...,Is hypertension a preventive measure?,cardiology
3,diabetes in children,diabetes in children is a common condition.,Is diabetes in children?,pediatrics
4,symptoms of depression,depression symptoms are a common symptom of de...,Is depression a common symptom?,mental health
